In [ ]:
!pip install torch transformers x-transformers betacal

In [ ]:
# # Basic imports
import numpy as np  # For numerical computations and array manipulations
import pandas as pd  # For loading and handling time-series and static data
import sys
import importlib
import os
import time
from tqdm import tqdm
# PyTorch imports
import torch  # Core PyTorch library
import torch.nn as nn  # Neural network layers and loss functions
import torch.optim as optim  # Optimization algorithms
from torch.utils.data import Dataset, DataLoader  # Datasets and DataLoaders for batching
from torch.nn import Transformer, TransformerEncoderLayer  # Transformer modules

# #Tranformers import
from transformers import AutoTokenizer, AutoModel

from sklearn.metrics import accuracy_score, precision_recall_curve, auc, roc_auc_score


module_path = '/home/workspace/files/MilanK/Model1/final_models/code'
# Add the module's directory to the system path if it's not already present
if module_path not in sys.path:
    sys.path.append(module_path)
    
    

from generating_datasets_for_torch import *
from load_static_data import *
from PatientDataset import *
from generate_labels_dataframe_with_dataloader import *
from load_train_test_split import *
from model import *
from load_patient_list import *
from forward_loop import *
from fit import *
from validate import *

In [ ]:
train,val,test = load_train_test_data(
    train_filename = 'train_patient_list_orig.txt',                                   
    val_filename = 'val_patient_list_orig.txt',
    test_filename = 'test_patient_list.txt'
)



In [ ]:
train_dataset = PatientDataset(patient_list = train, min_window_min=15, step_min=15,max_window_min=15,
                             prediction_window_length=15)


val_dataset = PatientDataset(patient_list = val, min_window_min=15, step_min=15,max_window_min=15,
                             prediction_window_length=15)



In [ ]:
batch_size = 64
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,             # Number of worker processes
    #prefetch_factor=1,          # Number of batches to preload per worker
    #persistent_workers=True      # Keep workers alive across epochs
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0,
    #prefetch_factor=1,
    #persistent_workers=True
)


In [ ]:

model = run_exp(experiment_name = 'combined_model_simpler_demographics_simpler_embeddings',
            train_loader = train_loader,
            val_loader = val_loader,
            depth=2,
            heads=2,
            dynamic_dim_in=18,
            dynamic_dim_out=64,
            max_seq_len=924,
            attn_dropout=0,
            ff_dropout=0,
            batch_norm=False,
            static_input_dim=47,
            scalar_mlp_hidden_dim=256,
            ensemble_mlp_hidden_dim=512,
            output_dim=1,
            l1_lambda=0,
            l2_lambda=1e-4,
            lr=3e-5,
            pos_weight_value=30.0,
            epochs = 20,
            max_norm=10,
            patience=10,
            lr_patience=5,
            device="cpu",
            target_outcome = 'resp',
            training_mode = 'simpler_embeddings'
               )